In [38]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
import csv
from datetime import datetime,timedelta
import cv2
import scipy.io as sio

In [2]:
plate = 13
date_init = datetime(2020,7,1,19,57)
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(24)]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [3]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [6]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(f'Data/graph_{date}_{plate}_full_labeled.csv',
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [4]:
from_tip_growth_pattern=[]
for date in dates[:-1]:
    from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(get_path(date,plate,True,extension='_connection.csv'))))

In [ ]:
from_tip_growth_pattern=[]
for date in dates[:-1]:
    from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(f'Data/connection_{date}_{plate}.csv')))

In [5]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [6]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [7]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))

In [8]:
factor = 5
final_pictures = [compress_skeleton(skeletons[i],factor)>=1 for i in range(len(skeletons))]

In [9]:
connections = [c[0] for c in from_tip_growth_pattern]
growth_patterns = [c[1] for c in from_tip_growth_pattern]

In [10]:
growths = [{tip : sum([len(branch) for branch in growth_pattern[tip]]) for tip in growth_pattern.keys()} for growth_pattern in growth_patterns]

In [42]:
degree1_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)==1 and node not in nx_graph_clean[i+1].nodes] for i,nx_graph in enumerate(nx_graph_clean[:-1])]
# degree1_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)==3] for i,nx_graph in enumerate(nx_graph_clean[:-1])]
t=1
tp1=t+1
plot_t_tp1(degree1_nodes[t],degree1_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.close('all')

In [11]:
growing_tips=[[node for node in growths[i].keys() if growths[i][node]>=80] for i in range(len(growths))]

In [41]:
t=0
tp1=t+1
plot_t_tp1(growing_tips[t],growing_tips[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

In [18]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht,from_tip,pos_tm1):
    anastomosis=[]
    origins=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    number_anastomosis = 0
    def dist_branch(node,nx_graph,pos):
        mini=np.inf
        for edge in nx_graph.edges:
            pixel_list=nx_graph.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos[node]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos[node]),axis=1))
                if distance<mini:
                    mini=distance
        return(mini)
    def count_neighbors_is_from_root(equ_list,nx_graph,root):
        count=0
        for neighbor in nx_graph.neighbors(root):
            if neighbor in equ_list:
                count+=1
        return(count)
    for tip in tips:
#         print(tip)
        consequence = from_tip[tip]
        for node in consequence:
            if node in nx_grapht.nodes and nx_grapht.degree(node)>=3 and count_neighbors_is_from_root(consequence,nx_grapht,node)<2:
#                 if node==2753:
#                     print(count_neighbors_is_from_root(consequence,nx_grapht,node))
#                     print(list(nx_grapht.neighbors(node)))
                anastomosis.append(node)
                origins.append(tip)
                number_anastomosis+=1
        if tip not in nx_grapht.nodes and dist_branch(tip,nx_grapht,pos_tm1)<=30 and nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['weight']>=20:
            origins.append(tip)
            number_anastomosis+=1/2
    return(anastomosis,origins,number_anastomosis)

In [15]:
def pinpoint_anastomosis2(nx_graph_tm1,nx_grapht,from_tip,pos_tm1):
    anastomosis=[]
    origins=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    number_anastomosis = 0
    def dist_branch(node,nx_graph,pos):
        mini=np.inf
        for edge in nx_graph.edges:
            pixel_list=nx_graph.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos[node]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos[node]),axis=1))
                if distance<mini:
                    mini=distance
        return(mini)
    def count_neighbors_is_from_root(equ_list,nx_graph,root):
        count=0
        for neighbor in nx_graph.neighbors(root):
            if neighbor in equ_list:
                count+=1
        return(count)
    for tip in tips:
#         print(tip)
        consequence = from_tip[tip]
        for node in consequence:
            if node in nx_grapht.nodes and nx_grapht.degree(node)>=3 and count_neighbors_is_from_root(consequence,nx_grapht,node)<2:
#                 if node==2753:
#                     print(count_neighbors_is_from_root(consequence,nx_grapht,node))
#                     print(list(nx_grapht.neighbors(node)))
                anastomosis.append(node)
                origins.append(tip)
                number_anastomosis+=1
        if tip not in nx_grapht.nodes and nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['weight']>=20:
            origins.append(tip)
            number_anastomosis+=1/2
    return(anastomosis,origins,number_anastomosis)

In [21]:
def find_origin_tip(node,from_tip):
    for tip in from_tip.keys():
        if node in from_tip[tip]:
            return(tip)

In [16]:
anas_result=[pinpoint_anastomosis2(nx_graph_clean[i],nx_graph_clean[i+1], connections[i],poss[i]) for i in range (len(dates)-1)]

In [19]:
anas_result=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i],poss[i]) for i in range (len(dates)-1)]
anastomosiss=[ana_result[0] for ana_result in anas_result]
origins=[ana_result[1] for ana_result in anas_result]
number_an = [ana_result[2] for ana_result in anas_result]

In [22]:
t=1
tp1=t+1
plot_t_tp1(origins[t],anastomosiss[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],
           compress=5,time='4h',relabel_tp1=lambda node : find_origin_tip(node,connections[t]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [131]:
plt.close('all')

In [58]:
from sparse_util import dilate
node_interest=25
window=1000
pos_problem=poss[t][node_interest]
xbegin=pos_problem[0]-window
ybegin=pos_problem[1]-window
xend=pos_problem[0]+window
yend=pos_problem[1]+window
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])
kernel = np.ones((5,5),np.uint8)
skeletons_small_dilated=[cv2.dilate(skeleton.todense().astype(np.uint8),kernel,iterations = 1) for skeleton in skeletons_small]



In [59]:
plot_t_tp1(node_smalls[t],node_smalls[tp1],poss[t],poss[tp1],skeletons_small_dilated[t],skeletons_small_dilated[tp1],shift=(xbegin,ybegin),time=f't={4*tp1}h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
def make_sparse(dico):
    dim=(20800, 46000)
    skel = sparse.dok_matrix(dim, dtype=bool)
    for key in dico.keys():
        skel[key]=dico[key]
    return(skel)

In [38]:
skeleton_dilated_dilated=make_sparse(dilate(dilate(skeletons[0])))
skeleton_dilated=make_sparse(dilate(skeletons[0]))

In [40]:
from sparse_util import dilate,zhangSuen
skeleton_thinned=make_sparse(zhangSuen(dilate(dilate(skeletons[0]))))

772031 771717
774151 773239
7295 292
11 2
0 0


In [36]:
node_interest=5119
pos_problem=poss[t][node_interest]
xbegin=pos_problem[0]-500
ybegin=pos_problem[1]-500
xend=pos_problem[0]+500
yend=pos_problem[1]+500

In [41]:
plot_t_tp1([],[],poss[t],poss[tp1],skeleton_thinned[xbegin:xend,ybegin:yend].todense(),skeleton_dilated_dilated[xbegin:xend,ybegin:yend].todense(),shift=(xbegin,ybegin),time=f't={4*tp1}h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
interest_node = 1351
t=5
tp1=t+1
plot_t_tp1([interest_node],[interest_node],poss[t],poss[tp1],final_pictures[t],final_pictures[t],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
degree4_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)==4] for nx_graph in nx_graph_clean]

In [24]:
numberdegree4=[len(liste) for liste in degree4_nodes]

In [25]:
len(nx_graph_clean[0].edges())

2296

In [25]:
number_anastomosis_theory=[2+len(nx_graph.edges)-len(nx_graph.nodes)-numberdegree4[i] for i,nx_graph in enumerate(nx_graph_clean)]

In [26]:
number_anastomosis_spot = [number_anastomosis_theory[0]]+[number_anastomosis_theory[0]+np.cumsum(number_an)[i] for i in range(len(anastomosiss))] 

In [27]:
per_frame_anastomosis_theory=[number_anastomosis_theory[i+1]-number_anastomosis_theory[i] for i in range(len(number_anastomosis_theory)-1)]

In [28]:
per_frame_anastomosis_spot=[number_anastomosis_spot[i+1]-number_anastomosis_spot[i] for i in range(len(number_anastomosis_theory)-1)]

In [29]:
number_nodes=[len(nx_graph.nodes) for nx_graph in nx_graph_clean]

In [30]:
number_edges=[len(nx_graph.edges) for nx_graph in nx_graph_clean]

In [31]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')
ax.set_ylabel('total number of anastomosis')
theory,=ax.plot(number_anastomosis_theory[:-1])
theory.set_label('theory')
spotting,=ax.plot(number_anastomosis_spot[:-1])
spotting.set_label('frame spotting')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')
ax.set_ylabel('number anastomosis per frame')
theory,=ax.plot(per_frame_anastomosis_theory[:-1])
theory.set_label('theory')
spotting,=ax.plot(per_frame_anastomosis_spot[:-1])
spotting.set_label('frame spotting')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
plt.close('all')

In [33]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(number_nodes[:-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
plot_t_tp1([1093],[1093],poss[t],poss[tp1],final_pictures[tp1],final_pictures[tp1],compress=5,time='4h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
import cv2
i=10
for node in origins[i]:
    node_interest=node
    pos_problem=poss[i][node_interest]
    xbegin=pos_problem[0]-500 
    ybegin=pos_problem[1]-500
    xend=pos_problem[0]+500 
    yend=pos_problem[1]+500
    kernel = np.ones((5,5),np.uint8)
    skeleton_small1=skeletons[i][xbegin:xend,ybegin:yend]
    skeleton_small1=cv2.dilate(skeleton_small1.todense().astype(np.uint8),kernel,iterations = 1)
    skeleton_small2=skeletons[i+1][xbegin:xend,ybegin:yend]
    skeleton_small2=cv2.dilate(skeleton_small2.todense().astype(np.uint8),kernel,iterations = 1)
    plot_t_tp1(origins[i],anastomosiss[i],poss[i],poss[i+1],skeleton_small1,skeleton_small2,time='4h',relabel_tp1=lambda node : find_origin_tip(node,connections[t]), shift=(xbegin,ybegin))

In [73]:
plt.close('all')

In [ ]:
t=3
tp1=t+1
plot_t_tp1([2180],[2180],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [23]:
plot_t_tp1(degree4_nodes[t],degree4_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [35]:
final_pictures_growth = [np.log(make_growth_picture_per_tip(growth_pattern[i],poss[i],factor=500,max_growth=4000,per_tip=True,min_growth=0)[0]+1) for i in range (len(growth_pattern))]


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_pictures_growth[0])

In [27]:
final_pictures[2].shape

(41, 91)

In [37]:
pos_problem=poss[0][444]
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [31]:
import cv2
kernel = np.ones((5,5),np.uint8)


In [40]:
skeletons_small_dilated=[cv2.dilate(skeleton.todense().astype(np.uint8),kernel,iterations = 1) for skeleton in skeletons_small]

In [38]:
t=0
tp1=t+1

In [42]:
plot_t_tp1(node_smalls[tp1],node_smalls[tp1],poss[tp1],poss[tp1],skeletons_small_dilated[tp1],skeletons_small_dilated[tp1],shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save='',time=None):
    right = 0.90
    top = 0.90
    if len(save)>=1:
        fig=plt.figure(figsize=(14,12))
        size = 10
    else:
        fig = plt.figure()
        size = 5
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc=colors.rgb2hex((1,1,1)))
    bbox_props2 = dict(boxstyle="circle", fc=colors.rgb2hex((0.5,0.5,0.5)))
    ax.text(right, top, time,
        horizontalalignment='right',
        verticalalignment='bottom',
        transform=ax.transAxes,color='white')
    for node in node_list_t:
        t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
                    size=size,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
                        size=size,
                        bbox=bbox_props2)
    if len(save)>=1:
        plt.savefig(save)
        plt.close(fig)
    else:
        plt.show()

In [78]:
def dist(t0,t1):
    distances=[]
    for node in nx_graph_clean[t0].nodes:
        if nx_graph_clean[t0].degree(node)==3 and node in nx_graph_clean[t1].nodes:
            distances.append(np.linalg.norm(poss[t0][node]-poss[t1][node]))
    return(distances)

In [82]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')
ax.set_ylabel('shift')
theory,=ax.plot([np.mean(dist(0,i)) for i in range(1,len(dates))])
theory.set_label('theory')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
gteer=[realign2(i,0) for i in range(0,len(dates))]

R0= [[1.00000000e+00 4.85722573e-17]
 [5.55111512e-17 1.00000000e+00]] t0= [0. 0.]
R0= [[ 9.99999996e-01 -9.17568871e-05]
 [ 9.17568871e-05  9.99999996e-01]] t0= [ 1.4874148 -1.0702497]
R0= [[ 9.99999999e-01 -4.41015591e-05]
 [ 4.41015591e-05  9.99999999e-01]] t0= [ 1.15100828 -1.70343372]
R0= [[ 9.99999994e-01 -1.12582077e-04]
 [ 1.12582077e-04  9.99999994e-01]] t0= [ 1.21817213 -2.76465166]
R0= [[ 9.99999999e-01 -4.02273870e-05]
 [ 4.02273870e-05  9.99999999e-01]] t0= [ 0.5877606 -0.5075092]
R0= [[ 9.99999992e-01  1.23009900e-04]
 [-1.23009900e-04  9.99999992e-01]] t0= [-2.57584682  1.00197054]
R0= [[ 9.99999996e-01 -8.42081649e-05]
 [ 8.42081649e-05  9.99999996e-01]] t0= [ 1.52502071 -1.40625473]
R0= [[ 9.99999998e-01 -5.79521654e-05]
 [ 5.79521654e-05  9.99999998e-01]] t0= [ 1.13990989 -1.83564832]
R0= [[ 9.99999994e-01 -1.06664958e-04]
 [ 1.06664958e-04  9.99999994e-01]] t0= [ 3.16237515 -1.59036676]
R0= [[ 9.99999997e-01 -8.03892362e-05]
 [ 8.03892362e-05  9.99999997e-01]] t0= [ 

In [124]:
final_pictures = [compress_skeleton(make_sparse(gteer[i]),factor)>=1 for i in range(len(gteer))]

In [125]:
plt.close('all')

In [127]:
t=5
tp1=t+1
plot_t_tp1(origins[t],anastomosiss[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],
           compress=5,time='4h',relabel_tp1=lambda node : find_origin_tip(node,connections[t]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
def transform_skeleton(skeleton_doc,Rot,trans):
    transformed_skeleton={}
    transformed_keys = np.round(np.transpose(np.dot(Rot,np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int)
    i=0
    for pixel in list(transformed_keys):
        i+=1
        transformed_skeleton[(pixel[0],pixel[1])]=1
    return(transformed_skeleton)

def find_common_group_nodes(t0,t1):
    nodest0=[]
    nodest1=[]
    for node in nx_graph_clean[t0].nodes:
        if nx_graph_clean[t0].degree(node)==3 and node in nx_graph_clean[t1].nodes and nx_graph_clean[t1].degree(node)==3:
            nodest0.append(poss[t0][node])
            nodest1.append(poss[t1][node])
    return(nodest0,nodest1)

def realign2(ti0,ti1,convergence_threshold=2,window=500,maxdist=50,save=''):
    t0=np.array([0,0])
    R0=np.identity(2)
    listeA,listeB = find_common_group_nodes(ti0,ti1)
    H=np.dot(np.transpose(np.array(listeA)-np.mean(listeA,axis=0)),np.array(listeB)-np.mean(listeB,axis=0))
    U,S,V=np.linalg.svd(H)
    R=np.dot(V,np.transpose(U))
    t=np.mean(listeB,axis=0)-np.dot(R,np.mean(listeA,axis=0))
    R0=np.dot(R,R0)
    t0=t+t0
#     return({key : np.dot(R0,poss[ti0][key])+t0 for key in poss[ti0].keys()})
    print("R0=",R0,'t0=',t0)
    skeleton_transformed=transform_skeleton(skeletons[ti0],R0,t0)
    return(skeleton_transformed)
#     skeleton_transformed=dilate(skeleton_transformed)
#     skeleton_transformed=dilate(skeleton_transformed)
#     skeleton_transformed=zhangSuen(skeleton_transformed)
#     if len(save)>=0:
#         from_nx_to_tab(*generate_nx_graph_from_skeleton(skeleton_transformed)).to_csv(save+'_raw_aligned_skeleton.csv')
#         np.savetxt(save+'rot.txt',R0)
#         np.savetxt(save+'trans.txt',t0)
#     print("R0=",R0,'t0=',t0)
#     return(R0,t0)
#     return(skeleton_transformed)


In [35]:
from matplotlib import colors